In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

def load_data(data_path, categories):
    """
    Завантаження даних з директорій
    """
    data = []
    labels = []
    for category in categories:
        folder_path = os.path.join(data_path, category)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            data.append(img)
            labels.append(category)
    return np.array(data), np.array(labels)

def preprocess_labels(labels):
    """
    Перетворення міток на числовий і категоріальний формати
    """
    labels = np.array([0 if label == 'with_mask' else 1 for label in labels])
    return to_categorical(labels)

def normalize_data(data):
    """
    Нормалізація даних
    """
    data = np.array(data, dtype="float32")
    return data / 255.0

if __name__ == '__main__':
    data_path = '/Users/iWolfer/Desktop/Pycharm/homework(pds6)/Lessons 60+/66/data'
    categories = ['with_mask', 'without_mask']

    # Завантаження даних
    data, labels = load_data(data_path, categories)

    # Передпроцесинг міток і даних
    labels = preprocess_labels(labels)
    data = normalize_data(data)

    # Розділення даних на тренувальний та тестовий набори
    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)


In [ ]:


# Задаємо архітектуру моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Навчання моделі
model.fit(trainX, trainY, epochs=10, validation_split=0.1, batch_size=32)


In [1]:
def predict_mask(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.reshape(img, [1, 224, 224, 3])
    prediction = model.predict(img)
    
    return 'With Mask' if np.argmax(prediction) == 0 else 'Without Mask'

# Перевірка
image_path = '/Users/iWolfer/Desktop/Pycharm/homework(pds6)/Lessons 60+/66/data/without_mask/3.jpg'  # Вставте шлях до вашого зображення
print(predict_mask(image_path))
